In [1]:
from bayes_opt import BayesianOptimization

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from utils import MVTecDataset, evaluate_accuracy
from models import MVTecSimpleCNN, MVTecResNet

import time
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('-t', '--target', required=True, help='target class')
parser.add_argument('-c', '--no_cuda', required=False, default=None, help='which cuda')
parser.add_argument('--lr', default= 0.001, type=float , required=False, help='learning rate')
parser.add_argument('--no_epoch', default= 30, type= int, required=False, help='number of epochs')
parser.add_argument('--model', default= 'simpleCNN', required=True, help='simpleCNN, resnet18')
# parser.add_argument('--freeze', default= True, type=bool, required=False, help='freeze pre-trained weights')

args = parser.parse_args("--target pill --model resnet18 -c 2 --lr 0.001".split())

In [3]:
args

Namespace(lr=0.001, model='resnet18', no_cuda='2', no_epoch=30, target='pill')

In [4]:
target_class = args.target
model_type = args.model
# model_freeze = args.freeze

save_path = os.path.join('./saves_BO/', target_class)

In [5]:
if not os.path.exists(save_path):
    os.makedirs(save_path)

device_type='cuda'

if args.no_cuda is not None:
    device_type = 'cuda:'+str(args.no_cuda)

In [6]:
device = torch.device(device_type if torch.cuda.is_available() else 'cpu')

normal_list_dir = [os.path.join('./data/', target_class, 'train', 'good'), os.path.join('./data/', target_class, 'test', 'good')]

test_dir = os.path.join('./data/', target_class, 'test')
test_subfolders = next(os.walk(test_dir))[1]

abnormal_list_dir=[]

for item in test_subfolders:
    if item != 'good':
        abnormal_list_dir.append(os.path.join('./data/', target_class, 'test', item))

dataset = MVTecDataset(normal_list_dir, abnormal_list_dir)

val_num = int(len(dataset)*0.15)
test_num = int(len(dataset)*0.15)
train_num = len(dataset) - val_num - test_num

train_dataset, valid_dataset, test_dataset =random_split(dataset,[train_num, val_num, test_num])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


# learning_rate = args.lr
# num_epoch = args.no_epoch

In [7]:
class MVTecCNN_BO(nn.Module):
    def __init__(self, num_channel):
        super(MVTecCNN_BO, self).__init__()
        self.num_channel = num_channel
        self.conv1 = nn.Conv2d(3, num_channel, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(num_channel)
        self.conv2 = nn.Conv2d(num_channel, num_channel*2, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(num_channel*2)
        self.conv3 = nn.Conv2d(num_channel*2, num_channel*4, 5)
        self.bn3 = nn.BatchNorm2d(num_channel*4)
        self.conv4 = nn.Conv2d(num_channel*4, num_channel*8, 3)
        self.bn4 = nn.BatchNorm2d(num_channel*8)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(num_channel*8*5*5, 128)
        self.bn_fc1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))

        x = x.view(-1, self.num_channel*8*5*5)
        x = self.relu(self.bn_fc1(self.fc1(x)))
        x = self.sigmoid(self.fc2(x))

        return x

In [8]:
def train(lr, num_channel):
    net = MVTecCNN_BO(num_channel).to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    best_val_acc = 0.
    num_epoch = 15
    
    for epoch in range(num_epoch):
        loss_count=0
        loss_sum=0
        for idx, (img, label) in enumerate(train_loader):
            img = img.to(device)
            label = label.to(device, dtype=torch.float)
            label = label.view(-1,1)
            pred = net(img)

            optimizer.zero_grad()
            loss = criterion(pred, label)
            loss.backward()
            optimizer.step()

            loss_sum+=loss.item()
            loss_count+=1
            if idx%10==0:
                net.eval()
                val_acc = evaluate_accuracy(net, valid_loader, device)
                if val_acc > best_val_acc:
                    best_val_acc = val_acc                    
                net.train()
       
    return best_val_acc

In [9]:
def cnn_function(lr, num_channel):
    num_channel = int(8 + num_channel*54)   # min 8, max 64
    best_val_accuracy = train(lr, num_channel)    
    return best_val_accuracy

In [10]:
# Bounded region of parameter space
pbounds = {'lr': (1e-7, 0.1), 'num_channel':(0, 1)}

In [11]:
optimizer = BayesianOptimization(
    f=cnn_function,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

In [12]:
start = time.time()
optimizer.maximize(
    init_points=3,
    n_iter=50,
)
end = time.time()
print('Time consumed: ' , end - start)

|   iter    |  target   |    lr     | num_ch... |
-------------------------------------------------
|  1        |  0.7209   |  0.0417   |  0.7203   |
|  2        |  0.7442   |  1.154e-0 |  0.3023   |
|  3        |  0.7093   |  0.01468  |  0.09234  |
|  4        |  0.7209   |  0.1      |  0.3892   |
|  5        |  0.6512   |  1e-07    |  0.5093   |
|  6        |  0.6977   |  0.1      |  1.0      |
|  7        |  0.6279   |  1e-07    |  1.0      |
|  8        |  0.686    |  0.1      |  0.1785   |
|  9        |  0.6628   |  0.1      |  0.8264   |
|  10       |  0.686    |  0.1      |  0.0      |
|  11       |  0.6047   |  0.1      |  0.6277   |
|  12       |  0.5814   |  1e-07    |  0.0      |
|  13       |  0.6977   |  0.01484  |  0.4067   |
|  14       |  0.686    |  0.01926  |  0.2937   |
|  15       |  0.6279   |  1e-07    |  0.8342   |
|  16       |  0.5581   |  1e-07    |  0.186    |
|  17       |  0.7558   |  0.000384 |  0.2976   |
|  18       |  0.6977   |  0.1      |  0.4917   |


In [14]:
optimizer.max

{'target': 0.813953488372093,
 'params': {'lr': 0.0011623431160238205, 'num_channel': 0.29725476914104565}}

In [15]:
int(8 + 0.29725476914104565*54)

24

- original CNN accuracy 0.7674 (lr: 0.001, num_channel: 32)
- auto-hyperparameter tuning with BO 0.8139 (lr: 0.00116, num_channel: 24)

In [ ]:
 val_acc = evaluate_accuracy(best, valid_loader, device)